In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import operator
import json

f = open('data/wudi.json',)
wudi = json.load(f)
wudi = {int(k):v for k, v in wudi.items()}


f = open('data/mudi.json',)
mudi = json.load(f)
mudi = {int(k):v for k, v in mudi.items()}
f.close()

num_users = len(np.unique(np.array(list(wudi.keys()))))
num_materials = len(np.unique(np.array(list(mudi.keys()))))

In [2]:
!pip install tensorflow

  Using cached tensorflow-2.4.1-cp38-cp38-manylinux2010_x86_64.whl (394.4 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
     |████████████████████████████████| 1.0 MB 34 kB/s  eta 0:00:01
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached tensorflow_estimator-2.4.0-py2.py3-none-any.whl (462 kB)
  Using cached grpcio-1.32.0-cp38-cp38-manylinux2014_x86_64.whl (3.8 MB)
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached wrapt-1.12.1.tar.gz (27 kB)
  Using cached absl_py-0.12.0-py3-none-any.whl (129 kB)
  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached tensorboard-2.5.0-py3-none-any.whl (6.0 MB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached google_auth_oauthlib-0.4.4-py2.py3-none-any.whl (18 kB)
     |████████

In [5]:
EMBEDDING_SIZE = 50

class RecommenderNet(keras.Model):
    def __init__(self, num_users, num_materials, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_materials = num_materials
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.user_bias = layers.Embedding(num_users, 1)
        self.material_embedding = layers.Embedding(
            num_materials,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.material_bias = layers.Embedding(num_materials, 1)

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        material_vector = self.material_embedding(inputs[:, 1])
        material_bias = self.material_bias(inputs[:, 1])
        dot_user_material = tf.tensordot(user_vector, material_vector, 2)
        # Add all the components (including bias)
        x = dot_user_material + user_bias + material_bias
        # The sigmoid activation forces the rating to between 0 and 1
        return tf.nn.sigmoid(x)

In [7]:
M = RecommenderNet(num_users, num_materials, 50)
M.load_weights("models/material_rec")


In [8]:
##### Input #########

one_user_id=29606863  ### existing_wholeasler_id

#################


one_user_wholesaler=[]
for i in range(num_materials):
    one_user_wholesaler.append(one_user_id)
one_user_material=[]
for i in range(num_materials):
    one_user_material.append(i)

one_user=pd.DataFrame({'wholesaler':one_user_wholesaler})
one_user['material']=one_user_material
one_user["wholesaler"] = [wudi[x] for x in one_user["wholesaler"]]

key_list_1 = list(mudi.keys())
val_list_1 = list(mudi.values())

preds=[]
for i in range(len(one_user)):
    preds.append((float(M.predict(np.asarray([one_user.iloc[i]]))[0]), key_list_1[val_list_1.index(one_user['material'][i])])) 

preds
#preds=dict(sorted(preds.items(), key=operator.itemgetter(1), reverse=True)[:10])
#ratings_for_all = pd.DataFrame(preds)
#ratings_for_all



[(0.6181020140647888, 10946),
 (0.5619761943817139, 19898),
 (0.4448304772377014, 80176),
 (0.5047046542167664, 66989),
 (0.3300358057022095, 3337),
 (0.26382797956466675, 48859),
 (0.2748631238937378, 29777),
 (0.3940323293209076, 33335),
 (0.46175339818000793, 5983),
 (0.616642951965332, 3372),
 (0.647419810295105, 11574),
 (0.47689464688301086, 20614),
 (0.36287569999694824, 59874),
 (0.5091548562049866, 71317),
 (0.33247479796409607, 63311),
 (0.5707223415374756, 9974),
 (0.4362448453903198, 48537),
 (0.2369624674320221, 37961),
 (0.4070000946521759, 59847),
 (0.21210601925849915, 3338),
 (0.4629511535167694, 10947),
 (0.5301554203033447, 6013),
 (0.5470837354660034, 59848),
 (0.3875483572483063, 77313),
 (0.3209097385406494, 76623),
 (0.548585057258606, 10965),
 (0.2693726420402527, 66839),
 (0.3181021511554718, 10957),
 (0.28752395510673523, 59873),
 (0.6021204590797424, 3410),
 (0.3801168203353882, 55357),
 (0.4343496263027191, 16550),
 (0.30736756324768066, 16519),
 (0.32357814

In [9]:
preds.sort(reverse=True)
preds

[(0.647419810295105, 11574),
 (0.6181020140647888, 10946),
 (0.616642951965332, 3372),
 (0.6021204590797424, 3410),
 (0.5707223415374756, 9974),
 (0.5619761943817139, 19898),
 (0.548585057258606, 10965),
 (0.5470837354660034, 59848),
 (0.5301554203033447, 6013),
 (0.5212371945381165, 65660),
 (0.5091548562049866, 71317),
 (0.505244791507721, 11766),
 (0.5047046542167664, 66989),
 (0.4974870979785919, 74342),
 (0.48882806301116943, 61629),
 (0.48078930377960205, 73181),
 (0.47689464688301086, 20614),
 (0.4756702184677124, 80417),
 (0.47322773933410645, 66719),
 (0.46746787428855896, 67483),
 (0.4629511535167694, 10947),
 (0.4622862935066223, 70601),
 (0.46175339818000793, 5983),
 (0.457344114780426, 80125),
 (0.4515451490879059, 3530),
 (0.4448304772377014, 80176),
 (0.44266772270202637, 63480),
 (0.44047775864601135, 54632),
 (0.439988911151886, 81408),
 (0.4362483024597168, 66738),
 (0.4362448453903198, 48537),
 (0.4343496263027191, 16550),
 (0.43227723240852356, 81410),
 (0.430992692

In [13]:
preds.sort(reverse=True)
preds

[(0.7555131316184998, 11574),
 (0.7122623920440674, 10946),
 (0.7030124664306641, 3372),
 (0.6905271410942078, 3410),
 (0.6790671348571777, 9974),
 (0.6542243361473083, 10965),
 (0.648714542388916, 59848),
 (0.6330955028533936, 19898),
 (0.6294881105422974, 6013),
 (0.6152001619338989, 71317),
 (0.6049131155014038, 65660),
 (0.6040411591529846, 11766),
 (0.599413275718689, 80125),
 (0.588787853717804, 80417),
 (0.5865527391433716, 20614),
 (0.5858160853385925, 66989),
 (0.5853895545005798, 61629),
 (0.5849428176879883, 67483),
 (0.5733973383903503, 10947),
 (0.5721898674964905, 82315),
 (0.5718077421188354, 66719),
 (0.569577693939209, 74342),
 (0.5637511014938354, 73181),
 (0.5582906007766724, 63480),
 (0.5525990724563599, 5983),
 (0.5487868785858154, 70601),
 (0.546637773513794, 3530),
 (0.5449057817459106, 81408),
 (0.5440109968185425, 54632),
 (0.5357252359390259, 80176),
 (0.5315258502960205, 81410),
 (0.5287601351737976, 48537),
 (0.5274576544761658, 66738),
 (0.5191006660461426,